In [ ]:
import urllib.request as req
import zipfile
import re
import csv
import random
import os
from collections import OrderedDict

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
file_url = "https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip"
req.urlretrieve(file_url, "download.zip")
with zipfile.ZipFile('download.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [59]:
def run_sql(sql):
    cmd = "psql -U postgres -d work -c \"%s\"" % (re.sub('[\r\n\t ]+', ' ', sql))
    if False:
        print(cmd)
    os.system(cmd)

def load_data_from_file(table_name, file_name):
    cmd = "cat %s | psql -U postgres -d work -c \"SET CLIENT_ENCODING='LATIN1'; COPY %s FROM STDIN WITH CSV DELIMITER ',' QUOTE '\\\"'\"" % (file_name, table_name)
    if False:
        print(cmd)
    os.system(cmd)

def load_csv_to_postgres(table_name, file_name):
    cols = []
    with open(file_name, newline='') as csvfile:
        rdr = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in rdr:
            for col in line:
                col_name = re.sub('([a-z])([A-Z])', '\\1_\\2', col).lower()
                cols.append(col_name)
            break
    table_name = "import.%s" % table_name
    run_sql('DROP TABLE IF EXISTS %s; CREATE TABLE %s(%s)' % (table_name, table_name, ', '.join([d + ' TEXT' for d in cols])))
    load_data_from_file(table_name, file_name)
    run_sql("UPDATE %s SET %s" % (table_name, ', '.join(['%s=trim(%s)' % (d, d) for d in cols])))
    run_sql("UPDATE %s SET %s" % (table_name, ', '.join(['%s=CASE WHEN %s = \'\' THEN NULL ELSE %s END' % (d, d, d) for d in cols])))


def run_clean_script(table_name):
    sql = open("sql/clean_%s.sql" % table_name).read()
    run_sql(sql)


In [ ]:
file_name = "data/EXTR_LookUp.csv"
table_name = "king_county_lookup"
load_csv_to_postgres(table_name, file_name)
run_clean_script(table_name, "WHERE lutype <> 'LUType'")

In [ ]:
!psql -U postgres -d work --tuples-only -c "select 'DROP TYPE IF EXISTS king_county_type_' || lu_type || ';  CREATE TYPE king_county_type_' || lu_type || ' AS ENUM (' || array_to_string(array_agg(quote_literal(descr)  ORDER BY descr), ', ') || ');' from (SELECT DISTINCT lu_type, descr FROM property.king_county_lookup) t group by lu_type;" > create_king_county_enums.sql
!psql -U postgres -d work -f create_king_county_enums.sql

In [ ]:
file_name = "data/EXTR_RPSale.csv"
table_name = "king_county_sales"
sload_csv_to_postgres(table_name, file_name)
run_clean_script(table_name)

In [ ]:
file_name = "data/EXTR_ResBldg.csv"
table_name = "king_county_building"
load_csv_to_postgres(table_name, file_name)
run_clean_script(table_name)

In [60]:
file_name = "data/EXTR_AptComplex.csv"
table_name = "king_county_apt_complex"
load_csv_to_postgres(table_name, file_name)
run_clean_script(table_name)

In [ ]:
file_name = "data/EXTR_CommBldg.csv"
table_name = "king_county_comm_bldg"
load_csv_to_postgres(table_name, file_name)
run_clean_script(table_name)

In [ ]:
file_name = "data/EXTR_CondoUnit2.csv"
table_name = "king_county_condo_unit"
load_csv_to_postgres(table_name, file_name)
run_clean_script(table_name)

In [ ]:
file_name = "data/EXTR_Parcel.csv"
table_name = "king_county_parcel"
load_csv_to_postgres(table_name, file_name)
run_clean_script(table_name)

In [ ]:
file_name = "data/EXTR_CondoComplex.csv"
table_name = "king_county_condo_complex"
load_csv_to_postgres(table_name, file_name)
run_clean_script(table_name)

In [ ]:
file_name = "data/EXTR_RealPropApplHist_V.csv"
table_name = "king_county_appraisal"
load_csv_to_postgres(table_name, file_name)
run_clean_script(table_name)

In [ ]:
import sys
from osgeo import osr

def esriprj2standards(shapeprj_path):
   prj_file = open(shapeprj_path, 'r')
   prj_txt = prj_file.read()
   srs = osr.SpatialReference()
   srs.ImportFromESRI([prj_txt])
   print('Shape prj is: %s' % prj_txt)
   print('WKT is: %s' % srs.ExportToWkt())
   print('Proj4 is: %s' % srs.ExportToProj4())
   srs.AutoIdentifyEPSG()
   print('EPSG is: %s' % srs.GetAuthorityCode(None))

# esriprj2standards(sys.argv[1])

In [ ]:
esriprj2standards("data/parcel_address/parcel_address.prj")

In [ ]:
# !shp2pgsql -s 2285 -W latin1 -D parcel_address.shp import.king_county_parcel_geo > king_county_parcel_geo.sql
# !psql -U postgres -d work -f king_county_parcel_geo.sql
run_clean_script("king_county_parcel_geo")